# imports

In [ ]:
import pysubgroup as ps
import pandas as pd
import numpy as np
import import_ipynb
import pickle
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
usa_order_data = pd.read_pickle('AggregatedDataUSA.pkl')
eu_order_data = pd.read_pickle('AggregatedDataEU.pkl')
apa_order_data = pd.read_pickle('AggregatedDataAPA.pkl')


In [ ]:
usa_dms_df = pd.read_pickle('prediction_results_usa.pkl')
apa_dms_df = pd.read_pickle('prediction_results_apa.pkl')
eu_dms_df = pd.read_pickle('prediction_results_eu.pkl')

In [ ]:
# load inflation to get I0/I1
inflation_df = pd.read_csv('df_inflation.csv') 

# function imports

In [ ]:
from functions_definitions import calculate_max_increase 
from functions_definitions import rec_for_row
from functions_definitions import est_qty_per_row
from functions_definitions import profit_check_row

# Get features for price calculation

In [ ]:
eu_order_data['MaxIncrease'] = eu_order_data['CurrentGLP'].apply(calculate_max_increase)
usa_order_data['MaxIncrease'] = usa_order_data['CurrentGLP'].apply(calculate_max_increase)
apa_order_data['MaxIncrease'] = apa_order_data['CurrentGLP'].apply(calculate_max_increase)

In [ ]:
weird_prices_eu = eu_order_data[eu_order_data['MaxIncrease'] > 3]
weird_prices_usa = usa_order_data[usa_order_data['MaxIncrease'] > 3]
weird_prices_apa = apa_order_data[apa_order_data['MaxIncrease'] > 3]

In [ ]:
eu_last_glp_series = (
    eu_order_data
    .groupby('ItemNumber')['CurrentGLP']
    .last()                             # get the last array per item
    .apply(lambda arr: arr[-1] if isinstance(arr, (list, np.ndarray)) and len(arr) > 0 else np.nan)
    .rename('LastGLP')
    .reset_index()
)

In [ ]:
apa_last_glp_series = (
    apa_order_data
    .groupby('ItemNumber')['CurrentGLP']
    .last()                             # get the last array per item
    .apply(lambda arr: arr[-1] if isinstance(arr, (list, np.ndarray)) and len(arr) > 0 else np.nan)
    .rename('LastGLP')
    .reset_index()
)

In [ ]:
usa_last_glp_series = (
    usa_order_data
    .groupby('ItemNumber')['CurrentGLP']
    .last()                             # get the last array per item
    .apply(lambda arr: arr[-1] if isinstance(arr, (list, np.ndarray)) and len(arr) > 0 else np.nan)
    .rename('LastGLP')
    .reset_index()
)

In [ ]:
eu_last_q_series = (
    eu_order_data
    .groupby('ItemNumber')['CurrentQuantity']
    .last()                             # get the last array per item
    .apply(lambda arr: arr[-1] if isinstance(arr, (list, np.ndarray)) and len(arr) > 0 else np.nan)
    .round(0)
    .rename('LastQty')
    .reset_index()
)

In [ ]:
usa_last_q_series = (
    usa_order_data
    .groupby('ItemNumber')['CurrentQuantity']
    .last()                             # get the last array per item
    .apply(lambda arr: arr[-1] if isinstance(arr, (list, np.ndarray)) and len(arr) > 0 else np.nan)
    .round(0)
    .rename('LastQty')
    .reset_index()
)

In [ ]:
apa_last_q_series = (
    apa_order_data
    .groupby('ItemNumber')['CurrentQuantity']
    .last()                             # get the last array per item
    .apply(lambda arr: arr[-1] if isinstance(arr, (list, np.ndarray)) and len(arr) > 0 else np.nan)
    .round(0)
    .rename('LastQty')
    .reset_index()
)

In [ ]:
eu_merged_df = eu_dms_df.merge(eu_last_glp_series, on='ItemNumber', how='left')
usa_merged_df = usa_dms_df.merge(usa_last_glp_series, on='ItemNumber', how='left')
apa_merged_df = apa_dms_df.merge(apa_last_glp_series, on='ItemNumber', how='left')

In [ ]:
eu_merged_df = eu_merged_df.merge(eu_last_q_series, on='ItemNumber', how='left')
usa_merged_df = usa_merged_df.merge(usa_last_q_series, on='ItemNumber', how='left')
apa_merged_df = apa_merged_df.merge(apa_last_q_series, on='ItemNumber', how='left')

In [ ]:
eu_merged_df = eu_merged_df.merge(
    eu_order_data[['ItemNumber', 'MaxIncrease']],
    on='ItemNumber',
    how='left'
)

In [ ]:
usa_merged_df = usa_merged_df.merge(
    usa_order_data[['ItemNumber', 'MaxIncrease']],
    on='ItemNumber',
    how='left'
)

In [ ]:
apa_merged_df = apa_merged_df.merge(
    apa_order_data[['ItemNumber', 'MaxIncrease']],
    on='ItemNumber',
    how='left'
)

In [ ]:
eu_merged_df['Predicted PE'] = eu_merged_df['Predicted PE'].clip(-100, 100)
usa_merged_df['Predicted PE'] = usa_merged_df['Predicted PE'].clip(-100, 100)
apa_merged_df['Predicted PE'] = apa_merged_df['Predicted PE'].clip(-100, 100)

# Load inflation

Change the filtering to reflect the current inflation index and previous inflation index. 

In [ ]:
# Filter the dataframe for the specified date range
inflation_2h_2024 = inflation_df[(inflation_df['DATE'] >= '2024-07-01') & (inflation_df['DATE'] <= '2024-12-31')]

# Calculate the average of InflationIndex
i0 = inflation_2h_2024['InflationIndex'].mean()
i0

In [ ]:

inflation_1h_2025 = inflation_df[(inflation_df['DATE'] >= '2025-01-01') & (inflation_df['DATE'] <= '2025-07-01')]
# Calculate the average of InflationIndex
i1 = inflation_1h_2025['InflationIndex'].mean()
i1

In [ ]:
r = i1 / i0
switch_point = 1.0 / (1.0 - 2.0 * r**2)

# Compute Recommended Prices

In [ ]:
eu_merged_df['Recommended GLP'] = eu_merged_df.apply(rec_for_row, axis=1, args=(i0, i1, 0.01))
usa_merged_df['Recommended GLP'] = usa_merged_df.apply(rec_for_row, axis=1, args=(i0, i1, 0.01))
apa_merged_df['Recommended GLP'] = apa_merged_df.apply(rec_for_row, axis=1, args=(i0, i1, 0.01))

In [ ]:
eu_merged_df["RecommendationCategory"] = np.where(
    eu_merged_df['Predicted PE'] < switch_point,
    'Decrease',
    np.where(
        eu_merged_df['Predicted PE'] < 0,
        'Keep Inflation',
        'Raise'
    )
)

In [ ]:
usa_merged_df["RecommendationCategory"] = np.where(
    usa_merged_df['Predicted PE'] < switch_point,
    'Decrease',
    np.where(
        usa_merged_df['Predicted PE'] < 0,
        'Keep Inflation',
        'Raise'
    )
)

In [ ]:
apa_merged_df["RecommendationCategory"] = np.where(
    apa_merged_df['Predicted PE'] < switch_point,
    'Decrease',
    np.where(
        apa_merged_df['Predicted PE'] < 0,
        'Keep Inflation',
        'Raise'
    )
)

# Compute Uplift

In [ ]:
eu_merged_df['EstQty'] = eu_merged_df.apply(est_qty_per_row, axis=1, args=(i0, i1))
usa_merged_df['EstQty'] = usa_merged_df.apply(est_qty_per_row, axis=1, args=(i0, i1))
apa_merged_df['EstQty'] = apa_merged_df.apply(est_qty_per_row, axis=1, args=(i0, i1))

In [ ]:
usa_merged_df['EstQty'] = usa_merged_df.apply(est_qty_per_row, axis=1, args=(i0, i1))
usa_merged_df['Uplift'] = usa_merged_df.apply(profit_check_row, axis=1, args=())
usa_price_recs_safe = usa_merged_df[~usa_merged_df['ItemNumber'].isin(weird_prices_usa['ItemNumber'])]

In [ ]:
eu_merged_df['Uplift'] = eu_merged_df.apply(profit_check_row, axis=1, args=())
usa_merged_df['Uplift'] = usa_merged_df.apply(profit_check_row, axis=1, args=())
apa_merged_df['Uplift'] = apa_merged_df.apply(profit_check_row, axis=1, args=())

In [ ]:
eu_price_recs_safe = eu_merged_df[~eu_merged_df['ItemNumber'].isin(weird_prices_eu['ItemNumber'])]
usa_price_recs_safe = usa_merged_df[~usa_merged_df['ItemNumber'].isin(weird_prices_usa['ItemNumber'])]
apa_price_recs_safe = apa_merged_df[~apa_merged_df['ItemNumber'].isin(weird_prices_apa['ItemNumber'])]

In [ ]:
eu_price_recs_safe["NewTotal"] = eu_price_recs_safe['Recommended GLP'] * eu_price_recs_safe['EstQty']
eu_price_recs_safe["LastTotal"] = eu_price_recs_safe['LastGLP'] * eu_price_recs_safe['LastQty']
eu_price_recs_safe["PercUplift"] =  eu_price_recs_safe["Uplift"]/ eu_price_recs_safe["LastTotal"]*100
eu_price_recs_safe["PercUplift"] = eu_price_recs_safe["PercUplift"].clip(upper=300)
eu_price_recs_safe["UpliftClipped"] = eu_price_recs_safe["PercUplift"] * eu_price_recs_safe["LastTotal"] / 100
eu_price_recs_safe["NewTotalClipped"] = eu_price_recs_safe["LastTotal"] + eu_price_recs_safe["UpliftClipped"]
eu_price_recs_safe["EstQtyClipped"] = eu_price_recs_safe["NewTotalClipped"] / eu_price_recs_safe["Recommended GLP"]
eu_price_recs_safe["UpliftClipped"] = eu_price_recs_safe["UpliftClipped"].round(4)
eu_price_recs_safe["Uplift"] = eu_price_recs_safe["Uplift"].round(4)
eu_price_recs_safe["NewTotalClipped"] = eu_price_recs_safe["NewTotalClipped"].round(4)
eu_price_recs_safe["NewTotal"] = eu_price_recs_safe["NewTotal"].round(4)
eu_price_recs_safe["EstQtyClipped"] = eu_price_recs_safe["EstQtyClipped"].round(4)
eu_price_recs_safe["EstQty"] = eu_price_recs_safe["EstQty"].round(4)

In [ ]:
apa_price_recs_safe["NewTotal"] = apa_price_recs_safe['Recommended GLP'] * apa_price_recs_safe['EstQty']
apa_price_recs_safe["LastTotal"] = apa_price_recs_safe['LastGLP'] * apa_price_recs_safe['LastQty']
apa_price_recs_safe["PercUplift"] =  apa_price_recs_safe["Uplift"]/ apa_price_recs_safe["LastTotal"]*100
apa_price_recs_safe["PercUplift"] = apa_price_recs_safe["PercUplift"].clip(upper=300)
apa_price_recs_safe["UpliftClipped"] = apa_price_recs_safe["PercUplift"] * apa_price_recs_safe["LastTotal"] / 100
apa_price_recs_safe["NewTotalClipped"] = apa_price_recs_safe["LastTotal"] + apa_price_recs_safe["UpliftClipped"]
apa_price_recs_safe["EstQtyClipped"] = apa_price_recs_safe["NewTotalClipped"] / apa_price_recs_safe["Recommended GLP"]
apa_price_recs_safe["UpliftClipped"] = apa_price_recs_safe["UpliftClipped"].round(4)
apa_price_recs_safe["Uplift"] = apa_price_recs_safe["Uplift"].round(4)
apa_price_recs_safe["NewTotalClipped"] = apa_price_recs_safe["NewTotalClipped"].round(4)
apa_price_recs_safe["NewTotal"] = apa_price_recs_safe["NewTotal"].round(4)
apa_price_recs_safe["EstQtyClipped"] = apa_price_recs_safe["EstQtyClipped"].round(4)
apa_price_recs_safe["EstQty"] = apa_price_recs_safe["EstQty"].round(4)

In [ ]:
usa_price_recs_safe["NewTotal"] = usa_price_recs_safe['Recommended GLP'] * usa_price_recs_safe['EstQty']
usa_price_recs_safe["LastTotal"] = usa_price_recs_safe['LastGLP'] * usa_price_recs_safe['LastQty']
usa_price_recs_safe["PercUplift"] =  usa_price_recs_safe["Uplift"]/ usa_price_recs_safe["LastTotal"]*100
usa_price_recs_safe["PercUplift"] = usa_price_recs_safe["PercUplift"].clip(upper=300)
usa_price_recs_safe["UpliftClipped"] = usa_price_recs_safe["PercUplift"] * usa_price_recs_safe["LastTotal"] / 100
usa_price_recs_safe["NewTotalClipped"] = usa_price_recs_safe["LastTotal"] + usa_price_recs_safe["UpliftClipped"]
usa_price_recs_safe["EstQtyClipped"] = usa_price_recs_safe["NewTotalClipped"] / usa_price_recs_safe["Recommended GLP"]
usa_price_recs_safe["UpliftClipped"] = usa_price_recs_safe["UpliftClipped"].round(4)
usa_price_recs_safe["Uplift"] = usa_price_recs_safe["Uplift"].round(4)
usa_price_recs_safe["NewTotalClipped"] = usa_price_recs_safe["NewTotalClipped"].round(4)
usa_price_recs_safe["NewTotal"] = usa_price_recs_safe["NewTotal"].round(4)
usa_price_recs_safe["EstQtyClipped"] = usa_price_recs_safe["EstQtyClipped"].round(4)
usa_price_recs_safe["EstQty"] = usa_price_recs_safe["EstQty"].round(4)

# Save Results

In [ ]:
eu_price_recs_safe.to_excel('eu_processed_recommendation.xlsx', index=False)
usa_price_recs_safe.to_excel('usa_processed_recommendation.xlsx', index=False)
apa_price_recs_safe.to_excel('apa_processed_recommendation.xlsx', index=False) 